In [15]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StringType, StructField, IntegerType, FloatType
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Mintempuratures").getOrCreate()

In [16]:
#sử dụng spark.read.csv, nó đã tự động trả về một DataFrame
#header=False: Mặc định, PySpark sẽ coi dòng đầu tiên trong tệp CSV là một phần của dữ liệu và không phải là tiêu đề của các cột.
df1 = spark.read.csv("file:///SparkCourse/1800.csv", schema="stationID String, date int, measure_type STRING, temperature float", header=False)
df1.show()

+-----------+--------+------------+-----------+
|  stationID|    date|measure_type|temperature|
+-----------+--------+------------+-----------+
|ITE00100554|18000101|        TMAX|      -75.0|
|ITE00100554|18000101|        TMIN|     -148.0|
|GM000010962|18000101|        PRCP|        0.0|
|EZE00100082|18000101|        TMAX|      -86.0|
|EZE00100082|18000101|        TMIN|     -135.0|
|ITE00100554|18000102|        TMAX|      -60.0|
|ITE00100554|18000102|        TMIN|     -125.0|
|GM000010962|18000102|        PRCP|        0.0|
|EZE00100082|18000102|        TMAX|      -44.0|
|EZE00100082|18000102|        TMIN|     -130.0|
|ITE00100554|18000103|        TMAX|      -23.0|
|ITE00100554|18000103|        TMIN|      -46.0|
|GM000010962|18000103|        PRCP|        4.0|
|EZE00100082|18000103|        TMAX|      -10.0|
|EZE00100082|18000103|        TMIN|      -73.0|
|ITE00100554|18000104|        TMAX|        0.0|
|ITE00100554|18000104|        TMIN|      -13.0|
|GM000010962|18000104|        PRCP|     

In [17]:
minTemps = df1.filter(df1.measure_type == "TMIN")

stationTemps = minTemps.select("stationID","temperature")
stationTemps.show()


+-----------+-----------+
|  stationID|temperature|
+-----------+-----------+
|ITE00100554|     -148.0|
|EZE00100082|     -135.0|
|ITE00100554|     -125.0|
|EZE00100082|     -130.0|
|ITE00100554|      -46.0|
|EZE00100082|      -73.0|
|ITE00100554|      -13.0|
|EZE00100082|      -74.0|
|ITE00100554|       -6.0|
|EZE00100082|      -58.0|
|ITE00100554|       13.0|
|EZE00100082|      -57.0|
|ITE00100554|       10.0|
|EZE00100082|      -50.0|
|ITE00100554|       14.0|
|EZE00100082|      -31.0|
|ITE00100554|       23.0|
|EZE00100082|      -46.0|
|ITE00100554|       31.0|
|EZE00100082|      -75.0|
+-----------+-----------+
only showing top 20 rows



In [18]:
minTempsByStation = stationTemps.groupBy("stationID").min("temperature")
minTempsByStation.show()

+-----------+----------------+
|  stationID|min(temperature)|
+-----------+----------------+
|ITE00100554|          -148.0|
|EZE00100082|          -135.0|
+-----------+----------------+



In [26]:
minTempsByStationF = minTempsByStation.withColumn("min(temperature)",(col("min(temperature)") - 32) * 0.1 * 5.0/9.0).sort("min(temperature)")
minTempsByStationF.show()
results = minTempsByStationF.collect()
for result in results:
    print(f"{result[0]} \t  {result[1]}")

+-----------+------------------+
|  stationID|  min(temperature)|
+-----------+------------------+
|ITE00100554|             -10.0|
|EZE00100082|-9.277777777777779|
+-----------+------------------+

ITE00100554 	  -10.0
EZE00100082 	  -9.277777777777779
